# Wapitify

This notebook serves to emphasize the utilization of the wapitify function, and we presume that the user has already reviewed the step-by-step wapiti tutorial.

In [7]:
from wapiti import wapitify, wapiti_tools

seaborn.set_context("notebook")
plt.rcParams["axes.grid"] = False
cwd = os.getcwd()  # current working directory

# Data preprocessing:

## /!\ 


It is assumed that the user possesses LBL per-line radial velocity (RV) timeseries, $\texttt{times_lbl}$, $\texttt{rvs_all}$, and $\texttt{drvs_all}$ as well as drift RVs. 

## /!\ 

Lines with less than 50% of spectra with a valid velocity are removed, and the data are subsequently binned by night.

In [5]:
g = np.mean(np.isfinite(rvs_all),axis=0)
valid = g >= 0.5
rvs_all, drvs_all = rvs_all[:, valid], drvs_all[:, valid]

In [8]:
rvs_binned = []
drvs_binned = []

for idx in tqdm(range(rvs_all.shape[1]), leave=False):

    time_binned, rv, drv = wapiti_tools.night_bin(times_lbl, rvs_all[:, idx]-rv_drift, np.sqrt(drvs_all[:, idx]**2+drv_drift**2))
    rvs_binned.append(rv)
    drvs_binned.append(drv)

rvs_binned = np.array(rvs_binned).T
drvs_binned = np.array(drvs_binned).T

  0%|          | 0/18047 [00:00<?, ?it/s]

It is feasible to apply the $\texttt{Wapiti}$ method directly by using the $\texttt{wapitify}$ function.

The function will return a dictionnary that encompasses all the products derived from the method.

In [9]:
results_wapiti = wapitify.wapitify(time_binned, rvs_binned, drvs_binned)
results_wapiti

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/175 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

...

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/154 [00:00<?, ?it/s]

{'time_used': array([2458471.01662653, 2458471.9971251 , 2458472.95502354,
        2458501.85148999, 2458738.14145446, 2458745.12416176,
        2458746.13130938, 2458751.13105619, 2458752.12272485,
        2458753.125943  , 2458759.1513477 , 2458763.13977478,
        2458766.1175354 , 2458769.13445571, 2458771.14359075,
        2458772.15598238, 2458773.1299771 , 2458788.12021982,
        2458789.12460369, 2458791.10408415, 2458792.14841385,
        2458793.13755229, 2458795.11102392, 2458797.06359375,
        2458798.15112502, 2458801.10130549, 2458802.07821616,
        2458823.03078351, 2458825.1023752 , 2458826.03472024,
        2458827.06536466, 2458829.09268654, 2458830.08387635,
        2458884.81609441, 2458895.79517331, 2458896.86848942,
        2458897.79633738, 2458898.83715015, 2458919.82519007,
        2458920.77210018, 2459112.11640603, 2459114.0510637 ,
        2459115.11574249, 2459116.10223236, 2459118.08669671,
        2459121.01891317, 2459122.1282684 , 2459123.02105

Although not advisable, it is also feasible to bypass the selection process for determining a reliable number of components by imposing a predetermined value chosen by the user.

In [10]:
results_wapiti = wapitify.wapitify(time_binned, rvs_binned, drvs_binned, n_components=2)
results_wapiti

  0%|          | 0/177 [00:00<?, ?it/s]

{'time': array([2458471.01662653, 2458471.9971251 , 2458472.95502354,
        2458501.85148999, 2458738.14145446, 2458745.12416176,
        2458746.13130938, 2458751.13105619, 2458752.12272485,
        2458753.125943  , 2458759.1513477 , 2458763.13977478,
        2458766.1175354 , 2458769.13445571, 2458771.14359075,
        2458772.15598238, 2458773.1299771 , 2458788.12021982,
        2458789.12460369, 2458791.10408415, 2458792.14841385,
        2458793.13755229, 2458795.11102392, 2458797.06359375,
        2458798.15112502, 2458801.10130549, 2458802.07821616,
        2458823.03078351, 2458825.1023752 , 2458826.03472024,
        2458827.06536466, 2458827.98740382, 2458829.09268654,
        2458830.08387635, 2458874.96007405, 2458875.96839383,
        2458876.97583548, 2458884.81609441, 2458885.93584124,
        2458895.79517331, 2458896.86848942, 2458897.79633738,
        2458898.83715015, 2458919.82519007, 2458920.77210018,
        2458922.7585227 , 2459112.11640603, 2459114.0510637 ,
